In [1]:
from IMPA.dataset.data_loader import CellDataLoader
from IMPA.solver import IMPAmodule
from IMPA.model import build_model

from omegaconf import OmegaConf
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os
import torch
from tqdm import tqdm
import pandas as pd
import seaborn as sns
from skimage import io, color, filters, measure
import yaml

import sys
sys.path.insert(0, "/home/icb/alessandro.palma/environment/IMPA/mol2image/models")
from proglow import build_proglow_model

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
path_to_configs = Path("/home/icb/alessandro.palma/environment/IMPA/IMPA/config_hydra/config")

**Read yaml files**

In [3]:
def count_IMPA_parameters(num_domains, single_style):
    args = {"img_size": 96, 
            "style_dim": 128,
            "n_channels": 6, 
            "dim_in": 64,
            "single_style": single_style,
            "stochastic": True, 
            "latent_dim": 512, 
            "z_dimension": 64, 
            "num_layers_mapping_net": 1}
    
    num_domains = num_domains
    args_dict = OmegaConf.create(args)
    model = build_model(args_dict, num_domains, "cuda")
    c = 0
    for m in model:
        c += count_params(model[m])
    del model
    # Free GPU memory
    torch.cuda.empty_cache()
    return c

def count_mol2image_parameters(num_domains):
    args_mol2image = {"modules": [{"type": "cond_proglowmpn", 
                "n_flow": 32, 
                "n_block": 1, 
                "in_channel": 6, 
                "img_size": 64, 
                "checkpoint": "/lustre/groups/ml01/workspace/alessandro.palma/imCPA_official/results/mol2img/karren_weights/64.pth", 
                "n_cond": 512}, 
                {"type": "cond_proglowmpn", 
                         "n_flow": 32,
                         "n_block": 1, 
                         "in_channel": 5, 
                         "img_size": 32, 
                         "checkpoint": "/lustre/groups/ml01/workspace/alessandro.palma/imCPA_official/results/mol2img/karren_weights/32.pth", "n_cond": 512}, 
                {"type": "cond_glowmpn", 
                         "n_flow": 32, 
                         "n_block": 2, 
                         "in_channel": 5, 
                         "img_size": 16, 
                         "checkpoint": "/lustre/groups/ml01/workspace/alessandro.palma/imCPA_official/results/mol2img/karren_weights/16.pth", "n_cond": 512}], 
                "z_shapes": [[15, 32, 32], [15, 16, 16], [10, 8, 8], [40, 4, 4]], 
                "temp": [1.0, 1.0, 1.0, 1.0]}
    net = build_proglow_model(args_mol2image["modules"], affine=True).to("cuda")
    net.freeze_base_blocks()
    return count_params(net)

In [4]:
def initialize_model_and_compute_memory(yaml_config):
    args_rdkit = OmegaConf.create(yaml_config)
    dataloader_rdkit = CellDataLoader(args_rdkit)
    solver_rdkit = IMPAmodule(args_rdkit, ".", dataloader_rdkit)
    return solver_rdkit

def count_params(model):
    n_params = sum(p.numel() for p in model.parameters())
    return n_params

Get the memory plot

In [5]:
results = {"number of parameters": [],
          "model": [],
          "number of molecules": []
         }

In [6]:
for number_of_domains in [5000]:
    results["number of parameters"] += [number_of_domains]*3
    # IMPA
    # c_IMPA = count_IMPA_parameters(number_of_domains, True)
    # starganv2
    c_starganv2 = count_IMPA_parameters(number_of_domains, False)
    # mol2image
    # c_mol2image = count_mol2image_parameters(number_of_domains)

In [ ]:
c_mol2image

In [7]:
c_starganv2

4653217678